# Fine-tuning Llama 3 8B for Network Security Expert

This notebook fine-tunes Llama 3 8B Instruct to become a specialized Network Security expert using Unsloth with QLoRA (4-bit quantization).

**Topics covered:**
- Firewall & Network Device Configuration (Cisco, Palo Alto, Fortinet)
- Cloud Security (AWS, Azure, GCP)
- Threat Detection & Incident Response (IDS/IPS, SIEM)

**Training Method:** QLoRA (4-bit quantization) for efficient training on consumer GPUs

## 1. Install Dependencies

Run this cell if packages aren't installed yet.

In [ ]:
# Install Unsloth and dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-wbh7fne7/unsloth_7b520af5442d40b5b51a1472cfc8f152
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-wbh7fne7/unsloth_7b520af5442d40b5b51a1472cfc8f152
  Resolved https://github.com/unslothai/unsloth.git to commit ab4061e106792fa91e1eba3e4f3d45fa8aba121e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached xformers-0.0.26.post1.tar.gz (4.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)


## 2. Load Model

Load Llama 3.1 8B Instruct with 4-bit quantization.

In [2]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048  # Can increase to 4096 if needed
dtype = None  # Auto-detect (float16 for T4, bfloat16 for Ampere+)
load_in_4bit = True  # Use 4-bit quantization for QLoRA

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.1-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"✓ Model loaded: Llama 3.1 8B Instruct (4-bit)")
print(f"✓ Max sequence length: {max_seq_length}")
print(f"✓ Data type: {dtype if dtype else 'Auto'}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

✓ Model loaded: Llama 3.1 8B Instruct (4-bit)
✓ Max sequence length: 2048
✓ Data type: Auto


## 3. Configure LoRA

Add LoRA adapters for parameter-efficient fine-tuning.

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # LoRA scaling
    lora_dropout = 0,  # 0 for QLoRA
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # Memory efficient
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✓ LoRA adapters configured")
print(f"  - Rank: 16")
print(f"  - Target modules: All attention and MLP layers")
print(f"  - Trainable parameters: ~{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.1f}M")

## 4. Load Dataset

Load your network security training data in ChatML/ShareGPT format.

In [ ]:
from datasets import load_dataset

# Load your dataset
# Replace with your actual dataset path
dataset_path = "../data/processed/network_security_qa.json"

try:
    dataset = load_dataset("json", data_files=dataset_path, split="train")
    print(f"✓ Dataset loaded: {len(dataset)} examples")

    # Show a sample
    print("\nSample conversation:")
    print("-" * 80)
    sample = dataset[0]['conversations']
    for msg in sample:
        role = msg['from']
        text = msg['value'][:200]
        print(f"{role.upper()}: {text}...\n")

except FileNotFoundError:
    print(f"❌ Dataset not found at {dataset_path}")
    print("\nPlease create your dataset first:")
    print("1. Run: python scripts/generate_synthetic_data.py --provider openai --topic cisco-firewall --count 50")
    print("2. Merge topic datasets into data/processed/network_security_qa.json")
    print("3. Validate: python scripts/validate_dataset.py data/processed/network_security_qa.json")
    raise

## 5. Format Dataset for Training

Apply Llama 3 chat template to the dataset.

In [ ]:
from unsloth.chat_templates import get_chat_template

# Apply Llama 3 chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    """Format conversations using chat template"""
    conversations = examples["conversations"]
    texts = []
    for convo in conversations:
        text = tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return {"text": texts}

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

print("✓ Chat template applied")
print("\nFormatted example (first 500 chars):")
print("-" * 80)
print(dataset[0]['text'][:500] + "...")

## 6. Configure Training

Set up training hyperparameters.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training configuration
training_args = TrainingArguments(
    # Output
    output_dir = "../outputs/network-security-lora",

    # Batch size & accumulation
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,  # Effective batch size = 8

    # Training duration
    num_train_epochs = 3,  # 1-3 epochs recommended
    # max_steps = 60,  # Use this for quick testing instead of epochs

    # Learning rate
    learning_rate = 2e-4,
    lr_scheduler_type = "linear",
    warmup_steps = 5,

    # Optimization
    weight_decay = 0.01,
    optim = "adamw_8bit",

    # Logging & saving
    logging_steps = 10,
    save_strategy = "steps",
    save_steps = 100,
    save_total_limit = 2,

    # Mixed precision
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),

    # Misc
    seed = 3407,
    report_to = "none",  # Disable wandb/tensorboard
)

print("✓ Training configuration:")
print(f"  - Epochs: {training_args.num_train_epochs}")
print(f"  - Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Learning rate: {training_args.learning_rate}")
print(f"  - Mixed precision: {'BF16' if training_args.bf16 else 'FP16'}")

## 7. Initialize Trainer

Create the SFTTrainer for supervised fine-tuning.

In [ ]:
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences
    args = training_args,
)

print("✓ Trainer initialized")
print(f"\nTraining {len(dataset)} examples")
print(f"Estimated training time: ~{len(dataset) * 3 / 3600:.1f} hours (very rough estimate)")

## 8. Start Training

Begin the fine-tuning process. Monitor the loss - it should decrease over time.

**Target loss:** 0.5-1.0 is generally good
**Red flags:**
- Loss not decreasing → adjust learning rate
- Loss near 0 → overfitting, reduce epochs

In [ ]:
# Start training
print("Starting training...")
print("Watch the loss values - they should decrease over time.\n")

trainer_stats = trainer.train()

print("\n" + "="*80)
print("✅ TRAINING COMPLETE!")
print("="*80)
print(f"Final loss: {trainer_stats.training_loss:.4f}")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")

## 9. Test the Model

Try out your fine-tuned model with some network security questions.

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Test questions
test_questions = [
    "How do I configure port security on a Cisco switch?",
    "Explain the difference between AWS Security Groups and Network ACLs.",
    "What Snort rules would detect SQL injection attempts?",
    "My VPN tunnel keeps dropping. How do I troubleshoot this?",
]

print("Testing the fine-tuned model...\n")
print("="*80)

for i, question in enumerate(test_questions, 1):
    print(f"\nTest {i}/{len(test_questions)}")
    print("-"*80)
    print(f"Question: {question}\n")

    # Format as chat
    messages = [
        {"role": "user", "content": question}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # Generate response
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

    # Decode and print
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # Extract just the assistant's response
    response = response.split("assistant\n\n")[-1] if "assistant" in response else response

    print(f"Answer:\n{response}")
    print("\n" + "="*80)

print("\n✅ Testing complete!")

## 10. Save the Model

Save the LoRA adapter (small ~100-200MB file).

In [ ]:
# Save locally
model.save_pretrained("../models/network-security-lora")
tokenizer.save_pretrained("../models/network-security-lora")

print("✓ Model saved locally to: models/network-security-lora")
print("\nOptional: Upload to Hugging Face Hub")
print("Uncomment and run the cell below to upload.")

In [ ]:
# Optional: Push to Hugging Face Hub
# Replace 'your-username' with your HF username

# model.push_to_hub("your-username/llama3-network-security-lora", token="YOUR_HF_TOKEN")
# tokenizer.push_to_hub("your-username/llama3-network-security-lora", token="YOUR_HF_TOKEN")

# print("✓ Model uploaded to Hugging Face Hub!")

## 11. Export to GGUF for Ollama

Convert to GGUF format for use with Ollama on your local machine.

In [ ]:
# First, save merged 16-bit model
print("Step 1: Merging LoRA with base model...")
model.save_pretrained_merged(
    "../models/merged-16bit",
    tokenizer,
    save_method="merged_16bit"
)
print("✓ Merged model saved\n")

# Convert to GGUF with multiple quantization levels
print("Step 2: Converting to GGUF format...")
print("This will create 3 quantized versions (Q4_K_M, Q5_K_M, Q8_0)\n")

model.save_pretrained_gguf(
    "../models/gguf",
    tokenizer,
    quantization_method=["q4_k_m", "q5_k_m", "q8_0"]
)

print("\n" + "="*80)
print("✅ GGUF CONVERSION COMPLETE!")
print("="*80)
print("\nCreated files:")
print("  - models/gguf/unsloth.Q4_K_M.gguf (~4.5GB) - Fastest, good quality")
print("  - models/gguf/unsloth.Q5_K_M.gguf (~5.5GB) - Balanced [RECOMMENDED]")
print("  - models/gguf/unsloth.Q8_0.gguf (~8GB) - Highest quality")
print("\nNext steps:")
print("1. Create Modelfile in models/ directory")
print("2. Run: ollama create network-security-expert -f models/Modelfile")
print("3. Test: ollama run network-security-expert")

## Summary

Congratulations! You've fine-tuned Llama 3 8B to be a Network Security expert.

### What You've Done:
1. ✅ Loaded Llama 3.1 8B Instruct with 4-bit quantization
2. ✅ Configured LoRA adapters for efficient training
3. ✅ Trained on your network security dataset
4. ✅ Tested the model with example questions
5. ✅ Saved the LoRA adapter
6. ✅ Converted to GGUF for Ollama deployment

### Next Steps:

**To use with Ollama:**
```bash
# 1. Create Modelfile (see models/Modelfile)
cd models
ollama create network-security-expert -f Modelfile

# 2. Run your model
ollama run network-security-expert
```

**To improve your model:**
- Generate more training data for weak areas
- Increase training epochs if underfitting
- Add more diverse scenarios and edge cases
- Combine with RAG for up-to-date CVE information

**Questions or issues?**
- Check the plan file for troubleshooting tips
- Review training loss curves
- Validate your dataset quality

🎉 Happy network securing!